# regression for speed
Author: Bujie Xu

speed = Distance/measured time

In [23]:
import numpy as np
import pandas as pd
import matplotlib
import sklearn
import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

In [6]:
# # learning curver
# from sklearn.model_selection import learning_curve
# from sklearn.model_selection import ShuffleSplit


# def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
#                         n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
#     """
#     Generate a simple plot of the test and training learning curve.

#     Parameters
#     ----------
#     estimator : object type that implements the "fit" and "predict" methods
#         An object of that type which is cloned for each validation.

#     title : string
#         Title for the chart.

#     X : array-like, shape (n_samples, n_features)
#         Training vector, where n_samples is the number of samples and
#         n_features is the number of features.

#     y : array-like, shape (n_samples) or (n_samples, n_features), optional
#         Target relative to X for classification or regression;
#         None for unsupervised learning.

#     ylim : tuple, shape (ymin, ymax), optional
#         Defines minimum and maximum yvalues plotted.

#     cv : int, cross-validation generator or an iterable, optional
#         Determines the cross-validation splitting strategy.
#         Possible inputs for cv are:
#           - None, to use the default 3-fold cross-validation,
#           - integer, to specify the number of folds.
#           - An object to be used as a cross-validation generator.
#           - An iterable yielding train/test splits.

#         For integer/None inputs, if ``y`` is binary or multiclass,
#         :class:`StratifiedKFold` used. If the estimator is not a classifier
#         or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

#         Refer :ref:`User Guide <cross_validation>` for the various
#         cross-validators that can be used here.

#     n_jobs : integer, optional
#         Number of jobs to run in parallel (default 1).
#     """
#     plt.figure()
#     plt.title(title)
#     if ylim is not None:
#         plt.ylim(*ylim)
#     plt.xlabel("Training examples")
#     plt.ylabel("Score")
#     train_sizes, train_scores, test_scores = learning_curve(
#         estimator, X, y, scoring="neg_mean_squared_error",cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
#     train_scores_mean = np.mean(train_scores, axis=1)
#     train_scores_std = np.std(train_scores, axis=1)
#     test_scores_mean = np.mean(test_scores, axis=1)
#     test_scores_std = np.std(test_scores, axis=1)
#     plt.grid()

#     plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
#                      train_scores_mean + train_scores_std, alpha=0.1,
#                      color="r")
#     plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
#                      test_scores_mean + test_scores_std, alpha=0.1, color="g")
#     plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
#              label="Training score")
#     plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
#              label="Cross-validation score")

#     plt.legend(loc="best")
#     return plt

In [7]:
# # validation curve
# from sklearn.model_selection import validation_curve

# def plot_validation_curve(estimator, title, X, y,cv=None, 
#                           n_jobs = 1,param_range=np.logspace(-6, -1, 100)):
#     train_scores, test_scores = validation_curve(
#         estimator, X, y, param_name="alpha", param_range=param_range,
#         cv=cv, scoring="neg_mean_squared_error", n_jobs = n_jobs )
#     train_scores_mean = np.mean(train_scores, axis=1)
#     train_scores_std = np.std(train_scores, axis=1)
#     test_scores_mean = np.mean(test_scores, axis=1)
#     test_scores_std = np.std(test_scores, axis=1)

#     plt.title(title)
#     plt.xlabel("alpha")
#     plt.ylabel("Score")
#     lw = 2
#     plt.semilogx(param_range, train_scores_mean, label="Training score",
#                  color="darkorange", lw=lw)
#     plt.fill_between(param_range, train_scores_mean - train_scores_std,
#                      train_scores_mean + train_scores_std, alpha=0.2,
#                      color="darkorange", lw=lw)
#     plt.semilogx(param_range, test_scores_mean, label="Cross-validation score",
#                  color="navy", lw=lw)
#     plt.fill_between(param_range, test_scores_mean - test_scores_std,
#                      test_scores_mean + test_scores_std, alpha=0.2,
#                      color="navy", lw=lw)
#     plt.legend(loc="best")
#     return plt

In [2]:
# load data
train_csv = pd.read_csv(filepath_or_buffer="../data/data_train.csv",sep=";")
test_csv = pd.read_csv(filepath_or_buffer="../data/data_test.csv",sep=";")

### Speed: predict result

In [3]:
speed_test = test_csv.loc[test_csv.Speed.isnull()]
speed_test.Speed = speed_test.Distance/speed_test.Measured_time

/Users/xbj/anaconda/lib/python2.7/site-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [4]:
speed_result = speed_test[['Id','Speed']]

In [5]:
speed_result.head()

,Id,Speed
0,1,10.134000
1,2,6.063448
2,3,12.037037
3,4,5.485714
4,5,6.646154


## Part2 :Regression for Candence

## Data Preprocessing (Cadence)

In [6]:
all_data = pd.concat((train_csv, test_csv))
all_data.Speed = all_data.Speed.fillna(all_data.Distance/all_data.Measured_time)

In [7]:
all_data.Date = pd.to_datetime(all_data.Date)
all_data.Time = pd.to_datetime(all_data.Time)
all_data['Year'] = all_data.Date.apply(lambda date:date.year)
all_data['Month'] = all_data.Date.apply(lambda date:date.month)
all_data['Hour'] = all_data.Time.apply(lambda time:time.hour)

In [8]:
# remove useless columns
cadence = all_data.drop(['Index','Date','Time'], axis=1)

In [9]:
# change categorical data to dummys
cadence.RiderID = cadence.RiderID.astype('category')
cadence.Month = cadence.Month.astype('category')
cadence.Hour = cadence.Hour.astype('category')
cadence.Year = cadence.Year.astype('category')
cadence = pd.get_dummies(cadence)

In [10]:
# add new feature speed/dis
cadence['1/Measured_time'] = 1/cadence.Measured_time
cadence['Speed^2'] = cadence.Speed ** 2
cadence['Speed^3'] = cadence.Speed ** 3

In [11]:
cadence.columns

Index([u'Average_Gradient', u'Average_heart_rate', u'Cadence', u'Distance',
       u'Highest_point', u'Id', u'Lowest_point', u'Max_Gradient',
       u'Max_heart_rate', u'Measured_time', u'Moving_time', u'Power', u'Speed',
       u'RiderID_1', u'RiderID_2', u'RiderID_3', u'RiderID_4', u'RiderID_5',
       u'RiderID_6', u'RiderID_7', u'RiderID_8', u'RiderID_9', u'RiderID_10',
       u'RiderID_11', u'RiderID_12', u'RiderID_13', u'RiderID_14',
       u'RiderID_15', u'Year_2008', u'Year_2009', u'Year_2010', u'Year_2011',
       u'Year_2012', u'Year_2013', u'Year_2014', u'Year_2015', u'Year_2016',
       u'Month_1', u'Month_2', u'Month_3', u'Month_4', u'Month_5', u'Month_6',
       u'Month_7', u'Month_8', u'Month_9', u'Month_10', u'Month_11',
       u'Month_12', u'Hour_0', u'Hour_1', u'Hour_2', u'Hour_3', u'Hour_4',
       u'Hour_5', u'Hour_6', u'Hour_7', u'Hour_8', u'Hour_9', u'Hour_10',
       u'Hour_11', u'Hour_12', u'Hour_13', u'Hour_14', u'Hour_15', u'Hour_16',
       u'Hour_17', u'Hour

In [12]:
cadence_train = cadence.dropna(subset=['Cadence','Power'],how = 'any')
cadence_test = cadence.loc[cadence['Cadence'].isnull()]

In [13]:
dropList = ['Cadence','Id']
cadence_X_train, cadence_y = cadence_train.drop(dropList, axis=1), cadence_train['Cadence']
cadence_columns = cadence_X_train.columns
cadence_X_test = cadence_test.drop(dropList, axis=1)

In [14]:
cadence_X_train.shape

(120000, 74)

In [307]:
# try log transform the target
# numeric_feats = cadence_train.dtypes[cadence_train.dtypes != "object"].index

#skewed_feats = cadence_train[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
#skewed_feats = skewed_feats[skewed_feats > 1]
#skewed_feats = skewed_feats.index
#skewed_feats = np.hstack((skewed_feats[1],skewed_feats[-1]))
#cadence_train[skewed_feats] = np.log1p(cadence_train[skewed_feats])
#cadence_train[['Distance','/Dist']] = np.log1p(cadence_train[['Distance','1/Dist']])

In [472]:
# from sklearn.preprocessing import MinMaxScaler
# min_max = MinMaxScaler()
# X_train = min_max.fit_transform(X_train)
# X_test = min_max.transform(X_test)

In [406]:
# from sklearn.preprocessing import StandardScaler
# x_scaler = StandardScaler()
# X_train = x_scaler.fit_transform(X_)

In [407]:
# X_train = np.hstack((X_train, riderID))

## Cadence: Models 

In [408]:
# from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV, Lasso
# from sklearn.model_selection import cross_val_score

# def rmse_cv(model):
#     rmse = np.sqrt(-cross_val_score(model, X_train, y, scoring = "neg_mean_squared_error", cv = 5))
#     return(rmse)

### 1. RandomForestRegressor

In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
rfr = RandomForestRegressor(n_estimators=1000, max_features=0.2)
# param_grid = {'n_estimators': [1000], 'max_features': [0.2]}
# cadence_model_randomForestRegressor = GridSearchCV(estimator=rfr, param_grid=param_grid, 
#                                                    n_jobs=-1, cv=5, verbose=20, scoring="neg_mean_squared_error")
cadence_model_randomForestRegressor.fit(cadence_X_train,cadence_y)

NameError: name 'n_es' is not defined

In [20]:
cadence_model_randomForestRegressor.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=0.2, max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [21]:
cadence_model_randomForestRegressor.best_score_

-116.33974846749757

In [22]:
cadence_model_randomForestRegressor.best_params_

{'max_features': 0.2, 'n_estimators': 1000}

### 2. Cadence: xgboost model:

In [29]:
help(xgb.XGBRegressor)

Help on class XGBRegressor in module xgboost.sklearn:

class XGBRegressor(XGBModel, sklearn.base.RegressorMixin)
 |  Implementation of the scikit-learn API for XGBoost regression.
 |      Parameters
 |  ----------
 |  max_depth : int
 |      Maximum tree depth for base learners.
 |  learning_rate : float
 |      Boosting learning rate (xgb's "eta")
 |  n_estimators : int
 |      Number of boosted trees to fit.
 |  silent : boolean
 |      Whether to print messages while running boosting.
 |  objective : string or callable
 |      Specify the learning task and the corresponding learning objective or
 |      a custom objective function to be used (see note below).
 |  nthread : int
 |      Number of parallel threads used to run xgboost.
 |  gamma : float
 |      Minimum loss reduction required to make a further partition on a leaf node of the tree.
 |  min_child_weight : int
 |      Minimum sum of instance weight(hessian) needed in a child.
 |  max_delta_step : int
 |      Maximum delta 

In [27]:
# import xgboost as xgb
# from sklearn import cross_validation
# cadence_X_dtrain, cadence_X_deval, cadence_y_dtrain, cadence_y_deval = \
#     cross_validation.train_test_split(cadence_X_train, cadence_y, random_state = 1026, test_size=0.3)
# dtrain = xgb.DMatrix(cadence_X_dtrain, cadence_y_dtrain)
# deval = xgb.DMatrix(cadence_X_deval, cadence_y_deval)
# watchlist = [(deval, 'eval')]
# params = {
#     'booster': 'gbtree',
#     'objective': 'reg:linear',
#     'subsample': 0.7,
#     'colsample_bytree': 0.85,
#     'eta': 0.01,
#     'max_depth': 7,
#     'seed': 2016,
#     'silent': 0,
#     'eval_metric': 'rmse'
# }
# clf = xgb.train(params, dtrain, 1000, watchlist, early_stopping_rounds=50)

### 3. Cadence: AdaBoost

In [21]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
rd = DecisionTreeRegressor(max_depth=7)
clf = AdaBoostRegressor(n_estimators=500,base_estimator=rd, learning_rate = 0.5, random_state= 2017)
scores = cross_val_score(clf, cadence_X_train, cadence_y, scoring = "neg_mean_squared_error", n_jobs=2)
# param_grid = {'n_estimators': [1000], 'max_features': [0.2]}
# cadence_model_randomForestRegressor = GridSearchCV(estimator=rfr, param_grid=param_grid, 
#                                                    n_jobs=-1, cv=5, verbose=20, scoring="neg_mean_squared_error")
# cadence_model_AdaBoost.fit(cadence_X_train,cadence_y)

In [22]:
scores

array([-653.71006308, -785.60322364, -513.625924  ])

### Predict (Cadence)

In [24]:
cadence_test.Cadence = cadence_model_randomForestRegressor.predict(cadence_X_test)
cadence_result = cadence_test[['Id','Cadence']]
cadence_result.head()

,Id,Cadence
30000,30001.0,73.4630
30001,30002.0,98.0581
30002,30003.0,93.4522
30003,30004.0,89.3409
30004,30005.0,71.1088


In [25]:
cadence_result.to_csv(path_or_buf="../submit/submit4_cadence.csv", sep=";",index=False)

# Part3: Power: Data Preprocessing

In [26]:
power = cadence

In [27]:
power.head()

,Average_Gradient,Average_heart_rate,Cadence,Distance,Highest_point,Id,Lowest_point,Max_Gradient,Max_heart_rate,Measured_time,...,Hour_17,Hour_18,Hour_19,Hour_20,Hour_21,Hour_22,Hour_23,1/Measured_time,Speed^2,Speed^3
0,0.0,105.8,85.3,9980.400,128.4,NaN,124.6,5.0,114,1375,...,0,0,0,0,0,0,0,0.000727,52.685426,382.415730
1,3.3,157.5,88.3,1666.010,204.6,NaN,147.9,16.0,170,252,...,0,0,0,0,0,0,0,0.003968,43.707315,288.955649
2,0.0,158.9,103.4,345.600,59.0,NaN,59.0,0.0,160,36,...,0,0,0,0,0,0,0,0.027778,92.160000,884.736000
3,-8.3,99.1,66.4,1572.900,617.8,NaN,487.2,-0.5,104,123,...,0,0,0,0,0,0,0,0.008130,163.527954,2091.163563
4,-7.6,100.3,60.3,435.352,54.4,NaN,21.4,-4.8,111,59,...,0,0,0,0,0,0,0,0.016949,54.447390,401.758984


In [28]:
power_train = power.dropna(subset=['Cadence','Power'],how = 'any')
power_test = power.loc[cadence['Power'].isnull()]

In [29]:
dropList = ['Power','Id']
power_X_train, power_y = power_train.drop(dropList, axis=1), power_train['Power']
columns = power_X_train.columns
power_X_test = power_test.drop(dropList, axis=1)

In [561]:
# from sklearn.preprocessing import MinMaxScaler
# min_max = MinMaxScaler()
# X_train = min_max.fit_transform(X_train)
# X_test = min_max.transform(X_test)

In [30]:
power_X_train.shape

(120000, 74)

## Power: Models

## Power: RandomForestRegressor

In [31]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
rfr = RandomForestRegressor()
param_grid = {'n_estimators': [1000], 'max_features': [0.2]}
power_model_randomForest = GridSearchCV(estimator=rfr, param_grid=param_grid, n_jobs=4, cv=5, verbose=20, scoring="neg_mean_squared_error")
power_model_randomForest.fit(power_X_train, power_y)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] max_features=0.2, n_estimators=1000 .............................
[CV] max_features=0.2, n_estimators=1000 .............................
[CV] max_features=0.2, n_estimators=1000 .............................
[CV] max_features=0.2, n_estimators=1000 .............................
[CV]  max_features=0.2, n_estimators=1000, score=-2649.451434, total=36.7min
[CV]  max_features=0.2, n_estimators=1000, score=-2317.414612, total=36.7min


[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed: 55.1min
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed: 55.1min remaining: 82.7min


[CV] max_features=0.2, n_estimators=1000 .............................
[CV]  max_features=0.2, n_estimators=1000, score=-1899.647764, total=36.7min


[Parallel(n_jobs=4)]: Done   3 out of   5 | elapsed: 61.0min remaining: 40.7min


[CV]  max_features=0.2, n_estimators=1000, score=-3262.600279, total=47.7min
[CV]  max_features=0.2, n_estimators=1000, score=-2089.771083, total=39.4min


[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed: 98.6min remaining:    0.0s
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed: 98.6min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'n_estimators': [1000], 'max_features': [0.2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=20)

In [32]:
power_model_randomForest.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=0.2, max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [33]:
power_model_randomForest.best_score_

-2443.7770342159642

In [34]:
power_model_randomForest.best_params_

{'max_features': 0.2, 'n_estimators': 1000}

### Power: Predict

In [35]:
power_X_test.head()

,Average_Gradient,Average_heart_rate,Cadence,Distance,Highest_point,Lowest_point,Max_Gradient,Max_heart_rate,Measured_time,Moving_time,...,Hour_17,Hour_18,Hour_19,Hour_20,Hour_21,Hour_22,Hour_23,1/Measured_time,Speed^2,Speed^3
15000,-0.4,114.8,83.1,9824.80,71.4,25.6,6.1,136,1906,1223,...,0,0,0,0,0,0,0,0.000525,26.570617,136.962750
15001,0.0,145.8,93.2,351.68,32.0,32.0,0.0,150,38,38,...,0,0,0,0,0,0,0,0.026316,85.650154,792.669636
15002,-3.7,117.0,89.6,1099.72,85.9,44.7,-0.6,137,136,136,...,0,0,0,0,0,0,0,0.007353,65.386250,528.724756
15003,1.7,121.4,93.0,574.70,22.0,8.1,9.7,126,75,75,...,0,0,0,0,0,0,0,0.013333,58.716460,449.924664
15004,0.0,112.9,93.5,1035.60,129.4,127.0,14.4,115,133,133,...,0,0,0,0,0,0,0,0.007519,60.629055,472.086088


In [36]:
def power_predict(model):
    power_test.Power = model.predict(power_X_test)
    power_result = power_test[['Id','Power']]
    return power_result
power_result = power_predict(power_model_randomForest)

In [37]:
power_result.head()

,Id,Power
15000,15001.0,120.3240
15001,15002.0,165.3472
15002,15003.0,83.2832
15003,15004.0,142.2309
15004,15005.0,105.3928


# Final Result

In [38]:
speed_result = speed_result.rename(columns={'Speed': 'Predictor'})
cadence_result = cadence_result.rename(columns = {'Cadence': 'Predictor'})
power_result = power_result.rename(columns={'Power': 'Predictor'})

In [39]:
result = pd.concat((speed_result,cadence_result,power_result))

In [40]:
result.Id = result.Id.astype("int")

In [41]:
result.to_csv(path_or_buf="../submit/submit4.csv", sep=";",index=False)